In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect("../data/Weather.db")

In [4]:
cursor = conn.cursor()

In [5]:
def pprint(records):
    for row in records:
        for ele in row:
            if type(ele) in (int, float, type(None)):
                print(str(ele).ljust(8), end='|')
            else:
                print(str(ele).ljust(28), end='|')
        print()

In [6]:
# Get all data with any old duplicate records removed
records = cursor.execute(
"""
SELECT id, datetime_added, datetime, is_measured, longitude, latitude, windSpeed, temperature, humidity, uvIndex
FROM Weather
GROUP BY datetime, longitude, latitude 
HAVING MAX(datetime_added);
""").fetchall()

pprint(records)

985     |2024-04-17 12:44:18.294501  |2024-04-16T13:00:00Z        |1       |-97.54  |25.94   |5.88    |24.69   |85      |0       |
1705    |2024-04-17 12:45:05.952504  |2024-04-16T13:00:00Z        |1       |-97.52  |25.9    |5.81    |24.63   |86      |0       |
1129    |2024-04-17 12:44:20.799510  |2024-04-16T13:00:00Z        |1       |-97.52  |25.94   |5.81    |24.69   |85      |0       |
409     |2024-04-17 12:44:08.423730  |2024-04-16T13:00:00Z        |1       |-97.48  |25.9    |5.81    |24.63   |87      |0       |
1273    |2024-04-17 12:44:23.214195  |2024-04-16T13:00:00Z        |1       |-97.48  |25.94   |5.81    |24.69   |85      |0       |
553     |2024-04-17 12:44:10.859364  |2024-04-16T13:00:00Z        |1       |-97.44  |25.9    |7.5     |25.31   |87      |0       |
1417    |2024-04-17 12:44:25.648118  |2024-04-16T13:00:00Z        |1       |-97.44  |25.94   |6.19    |24.81   |86      |0       |
1561    |2024-04-17 12:45:03.425427  |2024-04-16T13:00:00Z        |1       |-97.42 

In [50]:
# What's the latest temperature for all of the geo locations? What's the latest wind speed? 
records = cursor.execute(
"""
SELECT datetime, longitude, latitude, windSpeed, temperature
FROM Weather
WHERE is_measured = 1 -- not sure if we want latest actual or forecasted temp
GROUP BY longitude, latitude
HAVING MAX(datetime_added) and MAX(datetime);
""").fetchall()
pprint(records)

2024-04-17T12:00:00Z        |-97.54  |25.94   |4.19    |24.69   |
2024-04-17T12:00:00Z        |-97.52  |25.9    |4.13    |24.69   |
2024-04-17T12:00:00Z        |-97.52  |25.94   |4.13    |24.69   |
2024-04-17T12:00:00Z        |-97.48  |25.9    |3.81    |24.63   |
2024-04-17T12:00:00Z        |-97.48  |25.94   |4.0     |24.69   |
2024-04-17T12:00:00Z        |-97.44  |25.9    |3.19    |24.88   |
2024-04-17T12:00:00Z        |-97.44  |25.94   |3.81    |24.69   |
2024-04-17T12:00:00Z        |-97.42  |25.86   |2.81    |24.13   |
2024-04-17T12:00:00Z        |-97.4   |25.9    |2.88    |24.69   |
2024-04-17T12:00:00Z        |-97.38  |25.92   |3.19    |24.38   |


In [52]:
# What's the latest temperature for all of the geo locations? What's the latest wind speed? 
records = cursor.execute(
"""
SELECT datetime, longitude, latitude, windSpeed, temperature
FROM Weather
WHERE is_measured = 0 -- not sure if we want latest actual or forecasted temp
GROUP BY longitude, latitude
HAVING MAX(datetime_added) and MAX(datetime);
""").fetchall()
pprint(records)

2024-04-22T11:00:00Z        |-97.54  |25.94   |4.71    |21.22   |
2024-04-22T11:00:00Z        |-97.52  |25.9    |4.71    |21.22   |
2024-04-22T11:00:00Z        |-97.52  |25.94   |4.71    |21.22   |
2024-04-22T11:00:00Z        |-97.48  |25.9    |4.71    |21.22   |
2024-04-22T11:00:00Z        |-97.48  |25.94   |4.71    |21.22   |
2024-04-22T11:00:00Z        |-97.44  |25.9    |5.07    |21.31   |
2024-04-22T11:00:00Z        |-97.44  |25.94   |5.07    |21.31   |
2024-04-22T11:00:00Z        |-97.42  |25.86   |5.07    |21.31   |
2024-04-22T11:00:00Z        |-97.4   |25.9    |5.07    |21.31   |
2024-04-22T11:00:00Z        |-97.38  |25.92   |5.07    |21.31   |


In [59]:
# Show an hourly time series of temperature (or any other available weather variable) from a day ago to 5 days in the future for a selected location
longitude = -97.42
latitude = 25.86
records = cursor.execute(
"""
SELECT datetime, windSpeed, temperature, humidity, uvIndex
FROM Weather
WHERE longitude = ? and latitude = ?
GROUP BY datetime, longitude, latitude
HAVING MAX(datetime_added)
ORDER BY datetime;
""", (longitude, latitude)).fetchall()

pprint(records)

2024-04-16T13:00:00Z        |6.38    |24.31   |89      |0       |
2024-04-16T14:00:00Z        |8.81    |26.19   |76      |1       |
2024-04-16T15:00:00Z        |8.81    |27.5    |68      |3       |
2024-04-16T16:00:00Z        |8.69    |28.81   |63      |5       |
2024-04-16T17:00:00Z        |8.19    |29.81   |59      |8       |
2024-04-16T18:00:00Z        |8.0     |30.69   |54      |9       |
2024-04-16T19:00:00Z        |7.63    |31.38   |49      |8       |
2024-04-16T20:00:00Z        |6.88    |31.38   |53      |7       |
2024-04-16T21:00:00Z        |6.88    |30.81   |56      |5       |
2024-04-16T22:00:00Z        |6.81    |29.63   |62      |2       |
2024-04-16T23:00:00Z        |6.69    |28.31   |68      |0       |
2024-04-17T00:00:00Z        |6.19    |26.69   |76      |0       |
2024-04-17T01:00:00Z        |5.38    |25.63   |86      |0       |
2024-04-17T02:00:00Z        |5.63    |25.13   |91      |0       |
2024-04-17T03:00:00Z        |5.63    |25.31   |90      |0       |
2024-04-17